In [1]:
import time
from pathlib import Path
import requests
import json
import pandas as pd
import os 
import re 

import faiss

import os
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

from typing import List

In [2]:
# ! pip install pandas
# ! pip install faiss-cpu
# ! pip install sentence_transformers
# ! pip install einops

In [3]:
CHUNK_SIZE = 512
CHUNK_OVERLAP = 64

def chunk_text(text, size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    words = text.split()
    chunks = []
    for i in range(0, len(words), size - overlap):
        chunks.append(" ".join(words[i:i+size]))
    return chunks

In [4]:
# Setup paths
faiss_base_path = "./faiss-index-v1"
index_path = os.path.join(faiss_base_path, "faiss_index.index")
metadata_path = os.path.join(faiss_base_path, "faiss_metadata.json")

# Load model
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)  

index = faiss.IndexFlatL2(768) 
metadata_store = []

<All keys matched successfully>


In [5]:
doctype = "union-act"

In [6]:
%timeit
doctype_path_processed_metdata = os.path.join("documents-v1", "metadata")
doctype_path_processed_content = os.path.join("documents-v1", "content")

txt_files = [f for f in os.listdir(doctype_path_processed_content) if f.endswith('.txt')]

count = 0 
for t_file in txt_files: 
    
    print(f"Working on file {count}", end='\r')
    f_path_processed_metadata = os.path.join(doctype_path_processed_metdata, t_file.replace('.txt', '.json'))
    f_path_processed_content = os.path.join(doctype_path_processed_content, t_file)
    
    with open(f_path_processed_content, 'r', encoding='utf-8') as file:
        processed_text = file.read()
        
    with open(f_path_processed_metadata, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
            
    chunks = chunk_text(processed_text)

    embeddings = model.encode(chunks)
    index.add(np.array(embeddings))
    
    for i, chunk in enumerate(chunks):
        metadata_store.append({
            "source_file": t_file,
            "chunk_index": i,
            "text": chunk, 
            "doctype": doctype, 
            "title": metadata['title']
        })
        
    count += 1
    if count % 50 == 0: 
        faiss.write_index(index, index_path)
        with open(metadata_path, 'w') as f:
            json.dump(metadata_store, f, indent=2)

Working on file 1044

In [7]:
print(f"len(metadata) corresponding to {count} files: {len(metadata_store)}")

len(metadata) corresponding to 1045 files: 29181
